In [2]:
%pip install astpretty


In [13]:
import ast
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Modelo preentrenado para comprensión de código
MODEL_NAME = "Salesforce/codet5-small"  # Modelo preentrenado en código (CodeT5)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

def check_syntax(script):
    """
    Chequea la sintaxis del script usando ast.
    """
    try:
        ast.parse(script)
        return True, "No se detectaron errores de sintaxis."
    except SyntaxError as e:
        return False, f"Error de sintaxis: {e}"

def analyze_script_with_transformer(script):
    """
    Usa un modelo transformer para analizar el script y detectar posibles errores.
    """
    # Prepara el script como entrada para el modelo
    input_text = f"Detect errors in the following Python script:\n\n{script}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Genera predicciones del modelo
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction

def validate_python_script(script):
    """
    Valida un script de Python combinando AST y Transformer.
    """
    # Paso 1: Validación de sintaxis
    syntax_valid, syntax_message = check_syntax(script)

    # Paso 2: Validación con Transformer si la sintaxis es válida
    if syntax_valid:
        transformer_analysis = analyze_script_with_transformer(script)
        return f"Análisis del Script:\n{transformer_analysis}"
    else:
        return syntax_message

# Ejemplo de uso
script_with_error = """


def divide(a ,b):
    return  a/ b

prin((divide5,5)
"""

result = validate_python_script(script_with_error)
print(result)


Error de sintaxis: '(' was never closed (<unknown>, line 7)
